# MBDS - DSF - Final Project
## By Ennio Maldonado
### Whatami?!
A container deployment for a web application in flask that uses a convolutional neural network to classify images into 10 possible classes using the CIFAR-10 annotated data set and Keras.

### The DataSet

The CIFAR-10 and CIFAR-100 are labeled subsets of the 80 million tiny images dataset. They were collected by Alex Krizhevsky, Vinod Nair, and Geoffrey Hinton.
The CIFAR-10 dataset
The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.

Here are the classes in the dataset, as well as 10 random images from each:
![cifar10](cifar10.png)				

The classes are completely mutually exclusive. There is no overlap between automobiles and trucks. "Automobile" includes sedans, SUVs, things of that sort. "Truck" includes only big trucks. Neither includes pickup trucks. 

You can find some baseline replicable results on this dataset on the project page for cuda-convnet. These results were obtained with a convolutional neural network. Briefly, they are 18% test error without data augmentation and 11% with. Additionally, Jasper Snoek has a new paper in which he used Bayesian hyperparameter optimization to find nice settings of the weight decay and other hyperparameters, which allowed him to obtain a test error rate of 15% (without data augmentation) using the architecture of the net that got 18%.
Other results

### Dataset layout
#### Python / Matlab versions

I will describe the layout of the Python version of the dataset. The layout of the Matlab version is identical.

The archive contains the files data_batch_1, data_batch_2, ..., data_batch_5, as well as test_batch. Each of these files is a Python "pickled" object produced with cPickle. Here is a python2 routine which will open such a file and return a dictionary:

def unpickle(file):
    import cPickle
    with open(file, 'rb') as fo:
        dict = cPickle.load(fo)
    return dict

And a python3 version:

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

Loaded in this way, each of the batch files contains a dictionary with the following elements:

* data -- a 10000x3072 numpy array of uint8s. Each row of the array stores a 32x32 colour image. The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue. The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.

* labels -- a list of 10000 numbers in the range 0-9. The number at index i indicates the label of the ith image in the array data.


The dataset contains another file, called batches.meta. It too contains a Python dictionary object. It has the following entries:

* label_names -- a 10-element list which gives meaningful names to the numeric labels in the labels array described above. For example, label_names[0] == "airplane", label_names[1] == "automobile", etc.


### Imports

In [ ]:
# Import cifar10 from keras
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from skimage.transform import resize

%matplotlib inline

### Loading the Data

In [ ]:
# Data is already split into 50k train data and 10k test data so we only need to assing it.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# The data consist of 50k images that are 32 pixels heihgt, 32 pixels width and are RGB.
print('shape of features from training data:', x_train.shape)

In [ ]:
# The annotations consist of 50k labels (numebrs from 0 to 9 corresponding to the labels).
print('shape of target/labels from training data:', y_train.shape)

The labels are:

![labels](cifar20_labels.png)

In [ ]:
#Showing a sample of the images
im_size=15
im_rows = 5
im_col = 5
fig, axes = plt.subplots(im_rows,im_col,figsize=(im_size,im_size))
k=0
for i in range(5):
    for j in range(5):
        axes[i,j].imshow(x_train[k])
        axes[i,j].set_title(f'Label: {y_train[k]}',fontsize=im_size)
        axes[i,j].set_axis_off()
        k+=1
plt.show()

### The Model
I will be using 10 output neurons because the data has 10 classes, in order to be able to pass it to the model we need to have it in the format [0 0 0 0 0 0 0 0 0 1], for this will use keral utility to categorical in order to onehot encode the labels.

In [ ]:
# Function for encoding
def one_hot_encoding(dataset, classes):
    return keras.utils.to_categorical(dataset, classes)

In [ ]:
# Appliying one hot encoding
y_train_encoded = one_hot_encoding(y_train, 10)
y_test_encoded = one_hot_encoding(y_test, 10)

In [ ]:
print(f'The encoded label is: {y_train_encoded[0]}, and the normal label is:_ {y_train[0]}')

Finally neural networks are easier to train with normalized data, in order to normalize our data we will get the max and divide the data set by the mas, in this case since the images already go from 0 to 255, we only need to divide by 255.

In [ ]:
# Function for normalizing 
def normalize(dataset):
    return dataset.astype('float32') / x_train.max()   

In [ ]:
# Normalization 
x_train_norm = normalize(x_train)
x_test_norm = normalize(x_test)
head, *tail = x_train_norm
head

#### The Template / Archutecture of the CNN

Taken from josephlee94 on github.

ReLu activation for all layers except the output layer which is Softmax

* Conv Layer (Filter size 3x3, Depth 32) // The filter to predict based on parameters
* Conv Layer (Filter size 3x3, Depth 32) // Another layer
* Max Pool Layer (Filter size 2x2) // Get the max predicted value out of a 2 x 2 matrix
* Dropout Layer (Prob of dropout 0.25) // drop out layer for regularization
* Conv Layer (Filter size 3x3, Depth 64) // another layer
* Conv Layer (Filter size 3x3, Depth 64) // another layer
* Max Pool Layer (Filter size 2x2) // once again reduce dimentionality 
* Dropout Layer (Prob of dropout 0.25) // more regularization
* FC Layer (512 neurons) // Linear combinations 
* Dropout Layer (Prob of dropout 0.5) // normalization
* FC Layer, Softmax (10 neurons) // Output Layer Softmax - to prob distribution


In [ ]:
# I will using the sequential model
modelo_chingon = Sequential()

In [ ]:
# Layer 1: Convolutional Neuron, with depth = 32, filter = 3x3, 
# padding = same so that output has the same shape as the input.
# stride will be 1 the images are very small after all.
modelo_chingon.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32,32,3)))
# Layer 2: Same as layer 1, ecept we don't need to specfy shape anymore,
# is inferred from previous layer.
modelo_chingon.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
# Layer 3: Max Pool, filter = 2x2,
modelo_chingon.add(MaxPooling2D(pool_size=(2, 2)))
# Layer 4: Dropout, prob = 0.25 for regularization,
modelo_chingon.add(Dropout(0.25))
# Similarly for Layers 5 to 8, exceot the depth in the convolutional layer is of 64
modelo_chingon.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
modelo_chingon.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
modelo_chingon.add(MaxPooling2D(pool_size=(2, 2)))
modelo_chingon.add(Dropout(0.25))
# Flatten Layer to from Cube shape of depth 32 to a row.
modelo_chingon.add(Flatten())
# FC Layer of 512 Neurons
modelo_chingon.add(Dense(512, activation='relu'))
# Layer 10: Another dropout
modelo_chingon.add(Dropout(0.5))
# Output Layer with softmax activation
modelo_chingon.add(Dense(10, activation='softmax'))


In [ ]:
# Full summary of the layers:
modelo_chingon.summary()

#### Data Augmentation

In [ ]:
# Not implemented yet
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [ ]:
# Now to compile the model we use categorical cross entropy, which is good for many categories,
# the optimizer will be Adam which is a modified stochastic gradient descent,
# Finally we will track the accuracy of the model
modelo_chingon.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy','Precision', 'Recall'])

In [ ]:
# Train it!!
# Batch size of 32 with 20 epochs
# Trick to split 20% for validation at training step
hist = modelo_chingon.fit(x_train_norm, y_train_encoded, 
        batch_size=32, epochs=30,
        validation_split=0.2) 

In [ ]:
# Let's visualize the model loss and accuraccy

fig, axes = plt.subplots(1,4, figsize=(40,10))

axes[0].plot(hist.history['loss'])
axes[0].plot(hist.history['val_loss'])
axes[0].set_title('Model loss', fontsize= 40)
axes[0].set_ylabel('Loss', fontsize= 30)
axes[0].set_xlabel('Epoch', fontsize= 30)
axes[0].legend(['Train', 'Val'], loc='upper right')

axes[1].plot(hist.history['accuracy'])
axes[1].plot(hist.history['val_accuracy'])
axes[1].set_title('Model accuracy', fontsize= 40)
axes[1].set_ylabel('Accuracy', fontsize= 30)
axes[1].set_xlabel('Epoch', fontsize= 30)
axes[1].legend(['Train', 'Val'], loc='lower right')

axes[2].plot(hist.history['precision'])
axes[2].plot(hist.history['val_precision'])
axes[2].set_title('Model Precision', fontsize= 40)
axes[2].set_ylabel('Precision', fontsize= 30)
axes[2].set_xlabel('Epoch', fontsize= 30)
axes[2].legend(['Train', 'Val'], loc='lower right')

axes[3].plot(hist.history['recall'])
axes[3].plot(hist.history['val_recall'])
axes[3].set_title('Model Recall', fontsize= 40)
axes[3].set_ylabel('Recall', fontsize= 30)
axes[3].set_xlabel('Epoch', fontsize= 30)
axes[3].legend(['Train', 'Val'], loc='lower right')

for ax in axes:
    ax.tick_params(which='both', labelsize=25)
plt.show()

In [ ]:
# Finally time to pass it to the test data it has never seen.
modelo_chingon.evaluate(x_test_norm, y_test_encoded)[1]

In [ ]:
# Guardar el Modelo
modelo_chingon.save('cifar10_chingon.h5')

In [ ]:
model = keras.models.load_model('cifar10_chingon.h5')

In [2]:
from keras.models import load_model as lm   
from keras.preprocessing.image import img_to_array        
from keras.applications import imagenet_utils            
from PIL import Image                     
import numpy as np               
import flask                  
import io              

def load_model():                                                                         
    # load the pre-trained Keras model (here we are using a model                         
    # pre-trained on ImageNet and provided by Keras, but you can                          
    # substitute in your own networks just as easily)                                     
    global model                                                                          
    model = lm('cifar10_chingon.h5')       

def prepare_image(image, target):                              
    # if the image mode is not RGB, convert it             
    if image.mode != "RGB":                                          
        image = image.convert("RGB")                    
    # resize the input image and preprocess it           
    image = image.resize(target)              
    image = img_to_array(image)                            
    image = np.expand_dims(image, axis=0)                 
    image = imagenet_utils.preprocess_input(image)            
    # return the processed image                  
    return image                                  

In [5]:
from PIL import Image  
model = lm('cifar10_chingon.h5')
#Load the image
img = Image.open('cat.jpeg')
image = prepare_image(img, target=(32, 32))      
# classify the input image and then initialize the list       
# of predictions to return to the client                  
preds = model.predict(image)                 

                   
#for (imagenetID, label, prob) in results[0]:    
#    r = {"label": label, "probability": float(prob)}   
#    data["predictions"].append(r)             
#    # indicate that the request was a success         
#    data["success"] = True     
preds
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [6]:
def probabilities(preds, classes):
    return sorted(dict(zip(classes, *preds.tolist())).items(), key=lambda t: t[1], reverse=True)
    

In [13]:
probas = probabilities(preds, classes)

In [14]:
type(probas)

list

In [22]:
data = {"success": False} 
data["predictions"] = []                   
for (label, prob) in probas:    
    r = {"label": label, "probability": float(prob)}   
    data["predictions"].append(r)             
    # indicate that the request was a success         
    data["success"] = True   

In [23]:
data

{'success': True,
 'predictions': [{'label': 'cat', 'probability': 1.0},
  {'label': 'airplane', 'probability': 5.839614110306482e-28},
  {'label': 'automobile', 'probability': 0.0},
  {'label': 'bird', 'probability': 0.0},
  {'label': 'deer', 'probability': 0.0},
  {'label': 'dog', 'probability': 0.0},
  {'label': 'frog', 'probability': 0.0},
  {'label': 'horse', 'probability': 0.0},
  {'label': 'ship', 'probability': 0.0},
  {'label': 'truck', 'probability': 0.0}]}